In [1]:
%reset
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial import ConvexHull

import implicitmodules.torch as dm

from joblib import Memory

cache_directory = "./cache/"
memory = Memory(cache_directory, verbose=0)

torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
aabb_source = dm.Utilities.AABB(-1., 1., -0.5, 0.5)
density = 8
length_x = int(density*aabb_source.width)
length_y = int(density*aabb_source.height)
mesh_pts_x, mesh_pts_y = torch.meshgrid([torch.linspace(aabb_source.xmin, aabb_source.xmax, length_x), torch.linspace(aabb_source.ymin, aabb_source.ymax, length_y)])
pts_implicit1 = dm.Utilities.grid2vec(mesh_pts_x, mesh_pts_y)
source = pts_implicit1.clone()

In [3]:
# A polynomial model of order 3
def pol3(pos, abc):
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    g = abc[6].unsqueeze(1)
    h = abc[7].unsqueeze(1)
    i = abc[8].unsqueeze(1)
    j = abc[9].unsqueeze(1)

    return a \
        + b*pos[:, 0] + c*pos[:, 1] \
        + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1] \
        + g*pos[:, 0]**3 + h*pos[:, 1]**3 + i*pos[:, 0]**2*pos[:, 1] + j*pos[:, 0]*pos[:, 1]**2

my_pol = pol3

In [19]:
# The model parameters we use and would like to get back
abc_gt = torch.tensor([[1., 0.],
                       [4., 0.], [1., 1.],
                       [0., 1.], [0.5, 2.], [1., 1.],
                       [0., 0.1], [0.1, -0.2], [0., 0.5], [-0.2, -0.1]])
sigma_gt = 0.5
angle = 0.
# We create a target
@memory.cache
def generate_target(pts, abc, sigma, angle, seed):
    pts = torch.tensor(pts)
    abc = torch.tensor(abc)
    torch.manual_seed(seed)
    silent_dots_target = dm.DeformationModules.SilentLandmarks.build_from_points(pts)

    th = angle * math.pi * torch.ones(pts.shape[0])
    R_target = torch.stack([dm.Utilities.rot2d(t) for t in th])
    C_target = my_pol(pts, abc_gt).transpose(0, 1).unsqueeze(2)

    implicit1_target_pts_cotan = 3.*torch.rand_like(pts)
    implicit1_target_R_cotan = 3.*torch.rand_like(R_target)

    implicit1_target = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts.shape[0],
                             gd=(pts.view(-1).requires_grad_(),
                                 R_target.view(-1).requires_grad_()),
                             cotan=(implicit1_target_pts_cotan.clone().view(-1).requires_grad_(),
                                    implicit1_target_R_cotan.clone().view(-1).requires_grad_())),
        C_target, sigma_gt, 0.01, 1.)

    compound_target = dm.DeformationModules.CompoundModule([silent_dots_target,
                                                        implicit1_target])
    with torch.autograd.no_grad():
        dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound_target), 50, "midpoint")

    return silent_dots_target.manifold.gd.view(-1, 2)

target = generate_target(pts_implicit1.numpy(), abc_gt.numpy(), sigma_gt, angle, 1337)
target = target + 0.01*torch.randn_like(target)
aabb_target = dm.Utilities.AABB.build_from_points(target)

In [20]:
%matplotlib qt5

plt.axis('equal')
plt.plot(target.numpy()[:, 0], target.numpy()[:, 1], '.')
plt.plot(source.numpy()[:, 0], source.numpy()[:, 1], 'x')
plt.show()

In [6]:
# Callback function called when computing the energy of the model (before the actual shooting).
# Used to compute the C of the implicit module of order 1.
def parametric_c(modules, fitted_parameters):
    abc = fitted_parameters[-1]
    modules[1]._ImplicitModule1__C = my_pol(pts_implicit1, abc).transpose(0, 1).unsqueeze(2)


In [21]:
sigma = torch.cat([torch.linspace(0.01, 5., 100), torch.linspace(5.1, 50., 100)])
nu = 0.01
coeff = 1.

abc_init = torch.zeros(10, 2)
abc_init[0, 0] = 1.
abc_init[0, 1] = 1.
C_init = my_pol(pts_implicit1, abc_init).transpose(0, 1).unsqueeze(2)
C_target = my_pol(pts_implicit1, abc_gt).transpose(0, 1).unsqueeze(2)
th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(t) for t in th])

In [8]:
@memory.cache
def compute_c_dot(sigma, max_iter):
    abc = abc_init.clone().requires_grad_()
    implicit1 = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                             gd=(pts_implicit1.view(-1).requires_grad_(),
                                 R.view(-1).requires_grad_())),
        C_init, sigma, nu, coeff)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit1],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement(10.)],
        other_parameters=[abc],
        precompute_callback=parametric_c)

    fitter = dm.Models.ModelFittingScipy(model, 1., 1000.)
    fitter.fit([target], max_iter, log_interval=-1)

    C = model.modules[1].C.detach()
    return torch.dot((C/torch.norm(C)).contiguous().view(-1), (C_target/torch.norm(C_target)).contiguous().view(-1)).item()


In [9]:
max_iter = 250
c_dot = [compute_c_dot(s.item(), max_iter) for s in noise_sigma]

Initial energy = 627188.938


Optimization terminated successfully.
         Current function value: 627188.937500
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimisation process exited with message: Optimization terminated successfully.
Final energy = 627188.9375
Closure evaluations = 1
Time elapsed = 6.320948362350464
Jacobian min max =  0.0 0.0
Hessian condition number = 1.0


KeyboardInterrupt: 

In [ ]:
c_dot_stack = torch.stack(c_dot).tolist()
del c_dot_stack[40]
sigma_plot = sigma.tolist()
del sigma_plot[40]
print(sigma_plot)

plt.xlabel("$\sigma$")
plt.ylabel("$C_i \cdot C_\sigma$")
plt.plot(sigma_plot, c_dot_stack)
plt.show()